In [1]:
import json
import pystac

from benchmark import Timeseries
import sys
sys.path.append("..")
import helpers

In [2]:
%%capture
!pip install pystac

In [3]:
timechunks = "24"
bucket = "nasa-eodc-scratch"
directory = f"NLDAS/netcdf/.timechunk{timechunks}"
filename = f"stac_item_collection_{timechunks}.json"
catalog_url = f"s3://{bucket}/{directory}/{filename}"

In [4]:
credentials = helpers.get_credentials()
s3fsfs = helpers.create_s3filesystem(credentials)

In [5]:
with s3fsfs.open(catalog_url, 'r') as f:
    stac_json = json.load(f)

collection = pystac.ItemCollection.from_dict(stac_json)

In [6]:
collection

In [7]:
ts = Timeseries(
    collection=collection,
    latitude=slice(58,62),
    longitude=slice(-122,-118),
    start_date='2001-01-01T00:00:00.00',
    end_date='2001-01-01T23:00:00.00',
    fs=s3fsfs
)

In [8]:
%%time
ts.get_timeseries(variable='Tair')

2025-05-14 21:45:36,382 - benchmark - INFO - Starting timeseries generation for variable: Tair
2025-05-14 21:45:36,383 - benchmark - INFO - Processing STAC ItemCollection
2025-05-14 21:45:36,422 - benchmark - INFO - Found 1 matching assets
2025-05-14 21:45:36,423 - benchmark - INFO - Opening and combining datasets...
2025-05-14 21:45:59,544 - benchmark - INFO - Dataset opened successfully
2025-05-14 21:45:59,545 - benchmark - INFO - Slicing data for time range: 2001-01-01 00:00:00 to 2001-01-01 23:00:00
2025-05-14 21:45:59,549 - benchmark - INFO - Loading data into memory...
2025-05-14 21:46:13,576 - benchmark - INFO - Data loaded successfully
2025-05-14 21:46:13,577 - benchmark - INFO - Calculating mean across lat/lon dimensions...
2025-05-14 21:46:13,594 - benchmark - INFO - Timeseries generation complete


CPU times: user 4.18 s, sys: 2.15 s, total: 6.33 s
Wall time: 37.2 s


<xarray.DataArray 'Tair' (time: 24)> Size: 96B
array([261.18716, 261.1346 , 261.03973, 260.96863, 260.76938, 260.30664,
       259.64987, 259.05112, 258.63937, 258.4111 , 258.37787, 258.46387,
       258.62735, 258.79333, 258.843  , 258.62878, 258.12463, 257.90384,
       258.58868, 259.747  , 260.53784, 260.78662, 260.67273, 260.40457],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 192B 2001-01-01 ... 2001-01-01T23:00:00